In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import mlflow

# 1. DENEY AYARLARI
mlflow.set_experiment("Spoiler_Detection_DistilBERT")
MODEL_NAME = "distilbert-base-uncased"

# 2. ÖZEL VERİ SETİ SINIFI (PYTORCH İÇİN)
# Model veriyi direkt DataFrame olarak okuyamaz. 
# Bu sınıf, veriyi modelin anlayacağı "Tensor" formatına çevirir.
class SpoilerDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # O anki satırdaki veriyi (token ID'leri) al
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Eğer etiket varsa (Train aşamasıysa) onu da ekle
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# 3. BAŞARI METRİKLERİ HESAPLAMA FONKSİYONU
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1) # En yüksek olasılıklı sınıfı seç (0 veya 1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {'accuracy': acc, 'f1': f1}

def train():
    # Cihaz Seçimi 
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Device: Mac GPU (MPS) is being used! Fast training mode.")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("Device: NVIDIA GPU (CUDA) is being used!")
    else:
        device = torch.device("cpu")
        print("Device: CPU is being used. Training might be slow.")

    with mlflow.start_run():
        # VERİYİ YÜKLE
        print("Loading data...")
        try:
            df = pd.read_csv("data/cleaned_data.csv")
            df = df.dropna(subset=["cleaned_review_text", "label"])
        except FileNotFoundError:
            print("Error: Data file not found.")
            return

        X = df["cleaned_review_text"].tolist() # Listeye çevirmek şart
        y = df["label"].tolist()

        # Eğitim ve Test olarak böl
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # TOKENIZER (SÖZLÜK) YÜKLEME
        # DistilBERT'in kendi özel sözlüğünü indiriyoruz.
        print("Loading tokenizer...")
        tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

        # Metinleri sayıya çevir (Encoding)
        # truncation=True: Çok uzun yorumları kırp
        # padding=True: Kısa yorumları doldur (Hepsi eşit boy olsun)
        train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
        test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

        # Veriyi PyTorch formatına çevir
        train_dataset = SpoilerDataset(train_encodings, y_train)
        test_dataset = SpoilerDataset(test_encodings, y_test)

        # C. MODELİ YÜKLEME
        print("Downloading DistilBERT model...")
        model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
        model.to(device) # Modeli GPU'ya gönder

        # D. EĞİTİM AYARLARI (TRAINING ARGUMENTS)
        training_args = TrainingArguments(
            output_dir='./results',          # Checkpointleri buraya kaydet
            num_train_epochs=1,              # Şimdilik sadece 1 tur dönsün (Hız testi için)
            per_device_train_batch_size=16,  # Her adımda 16 yorum al
            per_device_eval_batch_size=16,
            warmup_steps=500,                # Isınma turları
            weight_decay=0.01,               # Ezberlemeyi önleme
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="epoch",           # Her tur sonunda test et 
            save_strategy="epoch",           # Her tur sonunda kaydet
            load_best_model_at_end=True,     # En iyi modeli sakla
        )

        # TRAINER (EĞİTMEN)
        # Hugging Face'in "Otomatik Pilotu". Döngüleri kendi kurar.
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            compute_metrics=compute_metrics
        )

        # EĞİTİMİ BAŞLAT
        print("Training starting! (This might take 5-10 mins depending on device speed)...")
        trainer.train()

        # SONUÇLARI GÖSTER VE KAYDET
        print("Evaluating on test set...")
        results = trainer.evaluate()
        
        print(f"\n--- RESULTS ---")
        print(f"Accuracy: {results['eval_accuracy']:.4f}")
        print(f"F1 Score: {results['eval_f1']:.4f}")

        # MLflow'a metrikleri kaydet
        mlflow.log_metric("eval_accuracy", results['eval_accuracy'])
        mlflow.log_metric("eval_f1", results['eval_f1'])
        
        # Modeli kaydet (MLflow Transformer desteği ile)
        # components={"tokenizer": tokenizer} sayesinde tokenizer'ı da içine paketler
        mlflow.transformers.log_model(
            transformers_model={"model": model, "tokenizer": tokenizer},
            artifact_path="distilbert_spoiler_model",
            task="text-classification"
        )
        print("\nModel saved to MLflow!")

if __name__ == "__main__":
    train()

🚀 Cihaz: Mac GPU (MPS) kullanılıyor! Hızlı eğitim modu.
Veri yükleniyor...
Tokenizer yükleniyor...
DistilBERT modeli indiriliyor...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Eğitim başlıyor! (Bu işlem bilgisayar hızına göre 5-10 dk sürebilir)...


/Users/alperen/Desktop/spoiler-detection-mlops/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.666800,0.670247,0.630000,0.669643


Test seti üzerinde değerlendiriliyor...


/Users/alperen/Desktop/spoiler-detection-mlops/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


2025/12/10 15:12:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Device set to use mps:0



--- SONUÇLAR ---
Accuracy: 0.6300
F1 Score: 0.6696

Model MLflow'a kaydedildi!
